In [1]:
# Block 0
# Import Libraries
from pynq import Overlay
from time import sleep
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_ADC
%matplotlib inline
import matplotlib.pyplot as plt
import struct

In [2]:
# Block 1
wave = list(range(64))
print(wave)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]


In [3]:
# Block 2
# Load RISCV2Core Processor
rv = Overlay("RV2Core.bit")
# Define Input Signal (AXI GPIO)
rstN = rv.rstN
step = rv.step
runAll = rv.runAll
hostCtrl = rv.hostCtrl
host_memWr = rv.host_memWr
host_memAdd = rv.host_memAdd
host_datToMem = rv.host_datToMem
# Define Output Signal (AXI GPIO)
datToHost = rv.datToHost
# Define Internal Signal/Variables
period = 2e-08 #20ns
ABI_Array = ['zero','ra','sp','gp','tp','t0','t1','t2','s0/fp','s1','a0','a1','a2','a3','a4','a5','a6','a7',
             's2','s3','s4','s5','s6','s7','s8','s9','s10','s11','t3','t4','t5','t6']
WRITE_VIEW = 1; #0 = No printing. 1 = Print everything
DATA_VIEW = 0; #0 = Print Everything. 1 = Print only Data Locations

In [5]:
# Block 3
WRITE_VIEW = WRITE_VIEW
IM0Array = []; IM1Array = []
for RVCore in range(0,2):
    string = ''; list = []; fullString = ''; data = []; strArray = ''; row = 0
    instrArray = []
    if RVCore == 0:
        file = open("swapX3DigitX4WithX5DigitX6_MachineCode.txt")
        string = file.read(); file.close
        instruction = string.split('\n')
    elif RVCore == 1:
        file = open("RV2Core1_MachineCode.txt")
        string = file.read(); file.close
        instruction = string.split('\n')
    for i in range(0,len(instruction)):
        if (len(instruction[i]) == 10):
            fullString = fullString + instruction[i] + '\n'
        elif(len(instruction[i]) == 8):
            fullString = fullString + "0x" + instruction[i] + '\n'
    for i in range(0,(64-len(instruction))):
            fullString = fullString + '0x00000000' + '\n'
    data = fullString.split('\n')
    for indx in range(0,len(data)-1):
        if indx == 63:
            strArray = strArray + data[indx]
        else:
            strArray = strArray + data[indx] + ", "
        row = row + 1
        if row == 8:
            if indx == 63:
                strArray = strArray
            else:
                strArray = strArray + "\n"
            row = 0     
    if(WRITE_VIEW == 1):
        IMArrayStr = 'IM' + str(RVCore) + 'Array = [' + strArray + ']'
        print(IMArrayStr, '\n')
        
    instrArray = strArray.split(", ")
    for i in instrArray:
        if RVCore == 0:
            IM0Array.append(int(i,16))
        elif RVCore == 1:
            IM1Array.append(int(i,16))
print("\nOperation Finished")

IM0Array = [0x0d0000ef, 0x00c000ef, 0x02c000ef, 0x04c000ef, 0x00f00393, 0x00000413, 0x00440863, 0x00439393, 
0x00140413, 0xfe441ce3, 0x0033f3b3, 0x0033c1b3, 0x00008067, 0x00f00493, 0x00000513, 0x00650863, 
0x00449493, 0x00150513, 0xfe651ce3, 0x0054f4b3, 0x0054c2b3, 0x00008067, 0x00620663, 0x00624a63, 
0x04625063, 0x0091e1b3, 0x0072e2b3, 0x00008067, 0x404305b3, 0x00000613, 0x0044d493, 0x00160613, 
0xfeb61ce3, 0x0091e1b3, 0x00000613, 0x00439393, 0x00160613, 0xfeb61ce3, 0x0072e2b3, 0x00008067, 
0x406205b3, 0x00000613, 0x00449493, 0x00160613, 0xfeb61ce3, 0x0091e1b3, 0x00000613, 0x0043d393, 
0x00160613, 0xfeb61ce3, 0x0072e2b3, 0x00008067, 0xb1ade1b7, 0x00700213, 0xb01df2b7, 0x0ce28293, 
0x00000313, 0x00008067, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000] 

IM1Array = [0x10000513, 0x00100593, 0x00000317, 0x034300e7, 0x00000317, 0x034300e7, 0x00000317, 0x030300e7, 
0x00000317, 0x030300e7, 0x00000317, 0x02c300e7, 0x00000317, 0x02c300e7, 0x0000006f, 0x00c52283, 
0xfe0

In [6]:
# Block 4
# Initialising all input signals
step.write(0,int('0',2));
runAll.write(0,int('0',2))
hostCtrl.write(0,int('0',2))
host_memWr.write(0,int('0',2))

host_memAdd_31_16 = int('0000000000000000',2)
host_memAdd_15 = int('0',2) #binary
host_memAdd_14_13 = int('00',2) #binary
host_memAdd_12_6 = int('0000000',2) #binary
host_memAdd_5_0 = int('000000',2) #binary

host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
host_memAdd.write(0,int(host_memAdd_str,2))
host_datToMem.write(0,0x00000000)
# Assert and Deassert ResetN.
rstN.write(0,int('0',2))
sleep(1.2*period)   # Period was defined as 20ns earlier.
rstN.write(0,int('1',2))
sleep(1.2*period)
print("\nOperation Finished")


Operation Finished


In [7]:
# Block 5
WRITE_VIEW = WRITE_VIEW
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2));
    host_memWr.write(0,int('1',2)); 
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('01',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nMemory of RVCore1\n")
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        host_datToMem.write(0,0x00000000)
        if(WRITE_VIEW == 1):
            print("Host Address:", hex(i), "\tData: 0x00000000")
        sleep(period)
    
    hostCtrl.write(0,int('0',2))
    host_memWr.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_14_13 = int('00',2) 
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
print("\nOperation Finished")

Memory of RVCore0

Host Address: 0x0 	Data: 0x00000000
Host Address: 0x1 	Data: 0x00000000
Host Address: 0x2 	Data: 0x00000000
Host Address: 0x3 	Data: 0x00000000
Host Address: 0x4 	Data: 0x00000000
Host Address: 0x5 	Data: 0x00000000
Host Address: 0x6 	Data: 0x00000000
Host Address: 0x7 	Data: 0x00000000
Host Address: 0x8 	Data: 0x00000000
Host Address: 0x9 	Data: 0x00000000
Host Address: 0xa 	Data: 0x00000000
Host Address: 0xb 	Data: 0x00000000
Host Address: 0xc 	Data: 0x00000000
Host Address: 0xd 	Data: 0x00000000
Host Address: 0xe 	Data: 0x00000000
Host Address: 0xf 	Data: 0x00000000
Host Address: 0x10 	Data: 0x00000000
Host Address: 0x11 	Data: 0x00000000
Host Address: 0x12 	Data: 0x00000000
Host Address: 0x13 	Data: 0x00000000
Host Address: 0x14 	Data: 0x00000000
Host Address: 0x15 	Data: 0x00000000
Host Address: 0x16 	Data: 0x00000000
Host Address: 0x17 	Data: 0x00000000
Host Address: 0x18 	Data: 0x00000000
Host Address: 0x19 	Data: 0x00000000
Host Address: 0x1a 	Data: 0x0000000

In [8]:
WRITE_VIEW = WRITE_VIEW

RVCore = 1
hostCtrl.write(0,int('1',2))
host_memWr.write(0,int('1',2))
bit = str(RVCore)
host_memAdd_15 = int(bit,2) 
host_memAdd_14_13 = int('01',2) 
host_memAdd_12_6 = int('0000000',2) 
host_memAdd_5_0 = int('000000',2) 
host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
host_memAdd.write(0,int(host_memAdd_str,2))
if RVCore == 0:
    print("Memory of RVCore0\n")
elif RVCore == 1:
    print("\nMemory of RVCore1\n")
for i in range(0,32):
    host_memAdd_5_0 = int('{0:06b}'.format(i),2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    data = wave[i];
    host_datToMem.write(0,data)
    if(WRITE_VIEW == 1):
        print("Host Address: ", hex(i),"\tData: ", hex(data));
    sleep(period)
host_memAdd_15 = int('0',2) #binary
host_memAdd_14_13 = int('00',2) #binary
host_memAdd_5_0 = int('000000',2) #binary
host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
host_memAdd.write(0,int(host_memAdd_str,2))
sleep(10*period)    
print("\nOperation Finished")


Memory of RVCore1

Host Address:  0x0 	Data:  0x0
Host Address:  0x1 	Data:  0x1
Host Address:  0x2 	Data:  0x2
Host Address:  0x3 	Data:  0x3
Host Address:  0x4 	Data:  0x4
Host Address:  0x5 	Data:  0x5
Host Address:  0x6 	Data:  0x6
Host Address:  0x7 	Data:  0x7
Host Address:  0x8 	Data:  0x8
Host Address:  0x9 	Data:  0x9
Host Address:  0xa 	Data:  0xa
Host Address:  0xb 	Data:  0xb
Host Address:  0xc 	Data:  0xc
Host Address:  0xd 	Data:  0xd
Host Address:  0xe 	Data:  0xe
Host Address:  0xf 	Data:  0xf
Host Address:  0x10 	Data:  0x10
Host Address:  0x11 	Data:  0x11
Host Address:  0x12 	Data:  0x12
Host Address:  0x13 	Data:  0x13
Host Address:  0x14 	Data:  0x14
Host Address:  0x15 	Data:  0x15
Host Address:  0x16 	Data:  0x16
Host Address:  0x17 	Data:  0x17
Host Address:  0x18 	Data:  0x18
Host Address:  0x19 	Data:  0x19
Host Address:  0x1a 	Data:  0x1a
Host Address:  0x1b 	Data:  0x1b
Host Address:  0x1c 	Data:  0x1c
Host Address:  0x1d 	Data:  0x1d
Host Address:  0x1e 	D

In [9]:
WRITE_VIEW = WRITE_VIEW
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('1',2))
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('00',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Write to IM0(Instruction Memory of RVCore0)\n")
    elif RVCore == 1:
        print("\nWrite to IM1(Instruction Memory of RVCore1)\n")    
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            host_datToMem.write(0,IM0Array[i])
            if(WRITE_VIEW == 1):
                print("Address: ",hex(int(host_memAdd_str,2)), "\t Host Address: ", hex(i), "\tData: ",hex(IM0Array[i]))
        elif RVCore == 1:
            host_datToMem.write(0,IM1Array[i])
            if(WRITE_VIEW == 1):
                print("Address: ",hex(int(host_memAdd_str,2)), "\t Host Address: ", hex(i), "\tData: ",hex(IM1Array[i]))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memWr.write(0,int('0',2))
    host_memAdd_15 = int('0',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    sleep(10*period)  
print("\nOperation Finished")

Write to IM0(Instruction Memory of RVCore0)

Address:  0x0 	 Host Address:  0x0 	Data:  0xd0000ef
Address:  0x1 	 Host Address:  0x1 	Data:  0xc000ef
Address:  0x2 	 Host Address:  0x2 	Data:  0x2c000ef
Address:  0x3 	 Host Address:  0x3 	Data:  0x4c000ef
Address:  0x4 	 Host Address:  0x4 	Data:  0xf00393
Address:  0x5 	 Host Address:  0x5 	Data:  0x413
Address:  0x6 	 Host Address:  0x6 	Data:  0x440863
Address:  0x7 	 Host Address:  0x7 	Data:  0x439393
Address:  0x8 	 Host Address:  0x8 	Data:  0x140413
Address:  0x9 	 Host Address:  0x9 	Data:  0xfe441ce3
Address:  0xa 	 Host Address:  0xa 	Data:  0x33f3b3
Address:  0xb 	 Host Address:  0xb 	Data:  0x33c1b3
Address:  0xc 	 Host Address:  0xc 	Data:  0x8067
Address:  0xd 	 Host Address:  0xd 	Data:  0xf00493
Address:  0xe 	 Host Address:  0xe 	Data:  0x513
Address:  0xf 	 Host Address:  0xf 	Data:  0x650863
Address:  0x10 	 Host Address:  0x10 	Data:  0x449493
Address:  0x11 	 Host Address:  0x11 	Data:  0x150513
Address:  0x12 	 H

In [11]:
hostCtrl.write(0,int('0',2))
host_memWr.write(0,int('0',2))
for i in range(0,300):
    step.write(0,int('1',2))
    sleep(period)
    step.write(0,int('0',2))
    sleep(period)
print("\nOperation Finished")


Operation Finished


In [10]:
IM0Dat = []
IM1Dat = []
DATA_VIEW = DATA_VIEW
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('0',2))
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('00',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Instruction Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nInstruction Memory of RVCore1\n")
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            IM0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            IM1Dat.append(hex(datToHost.read()))
        if(DATA_VIEW == 1):
            if(hex(datToHost.read()) != "0x0"):
                print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        elif(DATA_VIEW == 0):
            print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        sleep(period) 
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2) 
    host_memAdd_5_0 = int('000000',2) 
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")

Instruction Memory of RVCore0

Host Address: 0x0 	Data:  0xd0000ef
Host Address: 0x1 	Data:  0xc000ef
Host Address: 0x2 	Data:  0x2c000ef
Host Address: 0x3 	Data:  0x4c000ef
Host Address: 0x4 	Data:  0xf00393
Host Address: 0x5 	Data:  0x413
Host Address: 0x6 	Data:  0x440863
Host Address: 0x7 	Data:  0x439393
Host Address: 0x8 	Data:  0x140413
Host Address: 0x9 	Data:  0xfe441ce3
Host Address: 0xa 	Data:  0x33f3b3
Host Address: 0xb 	Data:  0x33c1b3
Host Address: 0xc 	Data:  0x8067
Host Address: 0xd 	Data:  0xf00493
Host Address: 0xe 	Data:  0x513
Host Address: 0xf 	Data:  0x650863
Host Address: 0x10 	Data:  0x449493
Host Address: 0x11 	Data:  0x150513
Host Address: 0x12 	Data:  0xfe651ce3
Host Address: 0x13 	Data:  0x54f4b3
Host Address: 0x14 	Data:  0x54c2b3
Host Address: 0x15 	Data:  0x8067
Host Address: 0x16 	Data:  0x620663
Host Address: 0x17 	Data:  0x624a63
Host Address: 0x18 	Data:  0x4625063
Host Address: 0x19 	Data:  0x91e1b3
Host Address: 0x1a 	Data:  0x72e2b3
Host Address: 0

In [11]:
MEM0Dat = []
MEM1Dat = []
DATA_VIEW = DATA_VIEW
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2)) 
    host_memWr.write(0,int('0',2)) 
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('01',2) #binary
    host_memAdd_12_7 = int('000000',2) #binary
    host_memAdd_6_0 = int('0000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
    if RVCore == 0:
        print("Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nMemory of RVCore1\n")
    for i in range(0,68):
        host_memAdd_6_0 = int('{0:07b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            MEM0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            MEM1Dat.append(hex(datToHost.read()))
        if(DATA_VIEW == 1):
            if(hex(datToHost.read()) != "0x0"):
                print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        elif(DATA_VIEW ==0):
            print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_6_0 = int('0000000',2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")

Memory of RVCore0

Host Address: 0x0 	Data:  0x0
Host Address: 0x1 	Data:  0x0
Host Address: 0x2 	Data:  0x0
Host Address: 0x3 	Data:  0x0
Host Address: 0x4 	Data:  0x0
Host Address: 0x5 	Data:  0x0
Host Address: 0x6 	Data:  0x0
Host Address: 0x7 	Data:  0x0
Host Address: 0x8 	Data:  0x0
Host Address: 0x9 	Data:  0x0
Host Address: 0xa 	Data:  0x0
Host Address: 0xb 	Data:  0x0
Host Address: 0xc 	Data:  0x0
Host Address: 0xd 	Data:  0x0
Host Address: 0xe 	Data:  0x0
Host Address: 0xf 	Data:  0x0
Host Address: 0x10 	Data:  0x0
Host Address: 0x11 	Data:  0x0
Host Address: 0x12 	Data:  0x0
Host Address: 0x13 	Data:  0x0
Host Address: 0x14 	Data:  0x0
Host Address: 0x15 	Data:  0x0
Host Address: 0x16 	Data:  0x0
Host Address: 0x17 	Data:  0x0
Host Address: 0x18 	Data:  0x0
Host Address: 0x19 	Data:  0x0
Host Address: 0x1a 	Data:  0x0
Host Address: 0x1b 	Data:  0x0
Host Address: 0x1c 	Data:  0x0
Host Address: 0x1d 	Data:  0x0
Host Address: 0x1e 	Data:  0x0
Host Address: 0x1f 	Data:  0x0
Host 

In [13]:
RB0Dat = []
RB1Dat = []
DATA_VIEW = DATA_VIEW
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('0',2))
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('10',2) #binary
    host_memAdd_12_5 = int('00000000',2) #binary
    host_memAdd_4_0 = int('00000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
    if RVCore == 0:
        print("Register Bank of RVCore0\n")
    elif RVCore == 1:
        print("\nRegister Bank of RVCore1\n")
    for i in range(0,32):
        host_memAdd_4_0 = int('{0:05b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
        host_memAdd.write(0,int(host_memAdd_str,2)) 
        if RVCore == 0:
            RB0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            RB1Dat.append(hex(datToHost.read()))
        regBankStr = "X"+"{:02d}".format(i)
        if(DATA_VIEW == 1):
            if(hex(datToHost.read()) != "0x0"):
                print(regBankStr, '  ', ABI_Array[i], '\tData: ',hex(datToHost.read()))
        elif(DATA_VIEW== 0):
            print(regBankStr, '  ', ABI_Array[i], '\tData: ',hex(datToHost.read()))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_4_0 = int('00000',2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")    

Register Bank of RVCore0

X00    zero 	Data:  0x0
X01    ra 	Data:  0x0
X02    sp 	Data:  0x0
X03    gp 	Data:  0x0
X04    tp 	Data:  0x0
X05    t0 	Data:  0x0
X06    t1 	Data:  0x0
X07    t2 	Data:  0x0
X08    s0/fp 	Data:  0x0
X09    s1 	Data:  0x0
X10    a0 	Data:  0x0
X11    a1 	Data:  0x0
X12    a2 	Data:  0x0
X13    a3 	Data:  0x0
X14    a4 	Data:  0x0
X15    a5 	Data:  0x0
X16    a6 	Data:  0x0
X17    a7 	Data:  0x0
X18    s2 	Data:  0x0
X19    s3 	Data:  0x0
X20    s4 	Data:  0x0
X21    s5 	Data:  0x0
X22    s6 	Data:  0x0
X23    s7 	Data:  0x0
X24    s8 	Data:  0x0
X25    s9 	Data:  0x0
X26    s10 	Data:  0x0
X27    s11 	Data:  0x0
X28    t3 	Data:  0x0
X29    t4 	Data:  0x0
X30    t5 	Data:  0x0
X31    t6 	Data:  0x0

Register Bank of RVCore1

X00    zero 	Data:  0x0
X01    ra 	Data:  0x0
X02    sp 	Data:  0x0
X03    gp 	Data:  0x0
X04    tp 	Data:  0x0
X05    t0 	Data:  0x0
X06    t1 	Data:  0x0
X07    t2 	Data:  0x0
X08    s0/fp 	Data:  0x0
X09    s1 	Data:  0x0
X10    a0 	

In [14]:
IMAdd = []
MEMAdd = []
RBAdd = []
addr = ''
for i in range(0,68):
    addr = "Host Address: "+hex(i)
    MEMAdd.append(addr) 
addr = ''
for i in range(0,64):
    addr = "Host Address: "+hex(i)
    IMAdd.append(addr) 
addr = ''
for i in range(0,32):
    regBankStr = "X"+"{:02d}".format(i)
    addr = regBankStr+','+ABI_Array[i]
    RBAdd.append(addr)
file = open("RB.txt", "w")
for i in range(0,32):
    if i == 31:
        file.write(repr(RBAdd[i])+","+repr(RB0Dat[i])+","+repr(RB1Dat[i])+"\n\n")
    elif i != 31:
        file.write(repr(RBAdd[i])+","+repr(RB0Dat[i])+","+repr(RB1Dat[i])+"\n")
file.close()
file = open("IM.txt", "w")
for i in range(0,64):
    if i == 63:
        file.write(repr(IMAdd[i])+","+repr(IM0Dat[i])+","+repr(IM1Dat[i])+"\n\n")
    elif i !=63:
        file.write(repr(IMAdd[i])+","+repr(IM0Dat[i])+","+repr(IM1Dat[i])+"\n")
file.close()
file = open("MEM.txt", "w")
for i in range(0,68):
    if i == 67:
        file.write(repr(MEMAdd[i])+","+repr(MEM0Dat[i])+","+repr(MEM1Dat[i])+"\n\n")
    elif i !=67:
        file.write(repr(MEMAdd[i])+","+repr(MEM0Dat[i])+","+repr(MEM1Dat[i])+"\n")
file.close()
print("\nOperation Finished")


Operation Finished


In [14]:
!cat main.c
!export PATH=/opt/riscv32/bin/:$PATH
!riscv32-unknown-elf-gcc -g main.c -march=r0i -mabi=ilp32 -o main
!riscv32-unknown-elf-objdump -d main >> CProgDump.s

int main(){
	int a = 2;
	int b = 2;
	int c = a + b;
}